In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import ta  
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.momentum import StochasticOscillator
from ta.trend import CCIIndicator
from ta.momentum import ROCIndicator
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange
from scipy.optimize import brute
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [5]:
symbols = ["BTCUSDT", "ETHUSDT", "XRPUSDT"]
data_path = "../data/"

### 1. RSI

In [6]:
# Grid search optimization for RSI parameters
def optimize_rsi_params(df):
    def objective(params):
        lookback, threshold_pct = params
        df_temp = df.copy()
        df_temp['rsi'] = RSIIndicator(df_temp['close'], window=int(lookback)).rsi()
        df_temp['momentum'] = df_temp['rsi'].diff(2)

        threshold = np.percentile(df_temp['momentum'].abs().dropna(), threshold_pct)
        df_temp['signal'] = np.where(df_temp['momentum'] > threshold, 1,
                                     np.where(df_temp['momentum'] < -threshold, -1, 0))

        returns = df_temp['signal'].shift(1) * df_temp['close'].pct_change()
        sharpe = returns.mean() / returns.std()
        return -sharpe

    ranges = (slice(3, 21, 2), slice(90, 99, 1))
    result = brute(objective, ranges, finish=None)
    return {'lookback': int(result[0]), 'percentile': result[1]}

# Generate signals with dynamic thresholding
def generate_signals(df, params):
    lookback = params['lookback']
    percentile = params['percentile']

    rsi = RSIIndicator(close=df['close'], window=lookback).rsi()
    df['momentum'] = rsi.diff(2)

    roll_window = max(6*60, lookback*20)
    df['threshold'] = df['momentum'].abs().rolling(roll_window).quantile(percentile / 100)

    cond_long = (df['momentum'] > df['threshold']) & (df['momentum'].shift(1) > df['threshold'].shift(1))
    cond_short = (df['momentum'] < -df['threshold']) & (df['momentum'].shift(1) < -df['threshold'].shift(1))

    df['signal'] = 0
    df.loc[cond_long, 'signal'] = 1
    df.loc[cond_short, 'signal'] = -1

    df['signal'] *= (df['volume'] > df['volume'].rolling(24*60).median())

    return df

# Apply ATR-based risk management
def apply_risk_management(df):
    atr_window = 14
    df['atr'] = AverageTrueRange(df['high'], df['low'], df['close'], window=atr_window).average_true_range()

    df['long_stop'] = (df['close'] - 2 * df['atr']).where(df['signal'] == 1).ffill()
    df['short_stop'] = (df['close'] + 2 * df['atr']).where(df['signal'] == -1).ffill()

    df['position'] = df['signal']
    df.loc[(df['close'] < df['long_stop']) & (df['signal'] == 1), 'position'] = 0
    df.loc[(df['close'] > df['short_stop']) & (df['signal'] == -1), 'position'] = 0

    return df

# Implement Kelly criterion for position sizing
def position_sizing(df):
    wins = df['position'] * df['close'].pct_change()
    win_rate = (wins > 0).mean()
    avg_win = wins[wins > 0].mean()
    avg_loss = -wins[wins < 0].mean()

    kelly_f = win_rate - (1 - win_rate) / (avg_win / avg_loss)
    max_position = 0.2

    volatility = df['close'].pct_change().rolling(24*60).std()
    df['position_size'] = np.minimum(kelly_f / volatility, max_position)

    return df

# Pipeline execution
final_results = {}

for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_3min_7days.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index()

    params = optimize_rsi_params(df)
    df = generate_signals(df, params)
    df = apply_risk_management(df)
    df = position_sizing(df)

    final_results[symbol] = df

# Example: Checking results
print(final_results['BTCUSDT'].tail())

                         open      high       low     close   volume  \
timestamp                                                              
2025-05-23 04:39:00  110564.3  110594.2  110507.3  110548.2  237.578   
2025-05-23 04:42:00  110548.2  110612.4  110538.1  110581.7  223.720   
2025-05-23 04:45:00  110581.7  110720.9  110572.2  110674.1  664.179   
2025-05-23 04:48:00  110674.2  110764.6  110661.9  110753.0  817.728   
2025-05-23 04:51:00  110753.0  110753.0  110716.2  110727.0   96.953   

                      momentum  threshold  signal         atr      long_stop  \
timestamp                                                                      
2025-05-23 04:39:00  -3.401173  16.724075       0  104.159671  110765.846346   
2025-05-23 04:42:00   5.632805  16.724075       0  102.026837  110765.846346   
2025-05-23 04:45:00  20.768801  16.781015       0  105.360634  110765.846346   
2025-05-23 04:48:00  24.091657  16.781015       1  105.170589  110542.658822   
2025-05-23 04:5

In [ ]:
lookbacks = [5, 7, 10, 14, 21]
percentiles = [90, 95, 97]  
vol_filter = True  

results_rsi = []

def preprocess_data(df):
    # remove the outliers
    df = df[(df['close'].pct_change().abs() < 0.05)]  # filter 5% change
    
    # dealing with the missing values
    df['volume'].replace(0, np.nan, inplace=True)
    df['volume'] = df['volume'].ffill()
    
    # adding volatility filter
    df['atr'] = AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=14).average_true_range()
    return df

def generate_signals(df, lookback, percentile):
    """信号生成优化"""
    # RSI calculation
    df['rsi'] = RSIIndicator(close=df['close'], window=lookback).rsi()
    
    # momentum improvement: using 3-period difference and normalizing
    df['rsi_momentum'] = df['rsi'].diff(3)
    df['rsi_momentum'] = (df['rsi_momentum'] - df['rsi_momentum'].rolling(24*60).mean()) / df['rsi_momentum'].rolling(24*60).std()
    
    # dynamic threshold calculation(rolling-window quantile)
    threshold = df['rsi_momentum'].abs().rolling(window=6*60).quantile(percentile/100)
    
    # signal generation
    cond_long = (df['rsi_momentum'] > threshold) & \
               (df['rsi_momentum'].shift(1) > threshold.shift(1))  # continuous 2-period upward trend
    cond_short = (df['rsi_momentum'] < -threshold) & \
                (df['rsi_momentum'].shift(1) < -threshold.shift(1))
    
    df['signal'] = 0
    df.loc[cond_long, 'signal'] = 1
    df.loc[cond_short, 'signal'] = -1
    
    # volatility filter
    if vol_filter:
        df['signal'] = df['signal'] * (df['atr'] < df['atr'].rolling(24*60).quantile(0.75)).astype(int)
    
    return df

def calculate_returns(df):
    # 
    df['return'] = df['close'].pct_change().shift(-1)
    
    # 止损逻辑（2倍ATR跟踪止损）
    df['long_stop'] = (df['close'] - 2*df['atr']).where(df['signal'] == 1).ffill()
    df['short_stop'] = (df['close'] + 2*df['atr']).where(df['signal'] == -1).ffill()
    
    # 生成有效信号
    df['position'] = df['signal']
    df.loc[(df['close'] < df['long_stop']) & (df['signal'] == 1), 'position'] = 0
    df.loc[(df['close'] > df['short_stop']) & (df['signal'] == -1), 'position'] = 0
    
    df['strategy_return'] = df['position'] * df['return']
    return df

for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_3min_7days.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index().pipe(preprocess_data)
    
    for lookback in lookbacks:
        for percentile in percentiles:  # 参数网格扩展
            df_temp = df.copy().pipe(generate_signals, lookback, percentile).pipe(calculate_returns)
            
            # 排除无信号时段
            valid_signals = df_temp[df_temp['position'] != 0]
            
            if not valid_signals.empty:
                hit_rate = (np.sign(valid_signals['position']) == np.sign(valid_signals['strategy_return'])).mean()
                sharpe = valid_signals['strategy_return'].mean() / valid_signals['strategy_return'].std()
                profit_factor = valid_signals[valid_signals['strategy_return'] > 0]['strategy_return'].sum() / \
                              abs(valid_signals[valid_signals['strategy_return'] < 0]['strategy_return'].sum())
            else:
                hit_rate = sharpe = profit_factor = 0
                
            results_rsi.append({
                "symbol": symbol,
                "lookback": lookback,
                "percentile": percentile,
                "hit_rate": round(hit_rate, 3),
                "sharpe_ratio": round(sharpe, 2),
                "profit_factor": round(profit_factor, 2),
                "signals": len(valid_signals),
                "avg_holding_bars": valid_signals['position'].diff().ne(0).cumsum().value_counts().mean()  # 平均持仓时间
            })

results_rsi_df = pd.DataFrame(results_rsi)
results_rsi_df

,symbol,lookback,percentile,hit_rate,sharpe_ratio,profit_factor,signals,avg_holding_bars
0,BTCUSDT,5,90,0.448,0.05,1.13,29,1.933333
1,BTCUSDT,5,95,0.400,-0.14,0.66,10,1.428571
2,BTCUSDT,5,97,0.500,-0.15,0.64,8,2.000000
3,BTCUSDT,7,90,0.469,0.13,1.40,32,1.882353
4,BTCUSDT,7,95,0.455,-0.05,0.88,11,1.571429
5,BTCUSDT,7,97,0.556,-0.05,0.87,9,2.250000
6,BTCUSDT,10,90,0.441,-0.08,0.81,34,2.266667
7,BTCUSDT,10,95,0.467,0.00,1.00,15,2.142857
8,BTCUSDT,10,97,0.444,-0.24,0.54,9,2.250000
9,BTCUSDT,14,90,0.459,-0.01,0.98,37,2.466667


In [3]:
lookbacks = [5, 7, 10, 14, 21]
percentile = 95

# Store all results
results_rsi = []

# Loop over symbols and lookback windows
for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_3min_7days.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index()

    for lookback in lookbacks:
        df_temp = df.copy()
        df_temp["rsi"] = RSIIndicator(close=df_temp["close"], window=lookback).rsi()
        df_temp["rsi_momentum"] = df_temp["rsi"].diff()

        threshold = np.percentile(df_temp["rsi_momentum"].abs().dropna(), percentile)

        df_temp["signal"] = 0
        df_temp.loc[df_temp["rsi_momentum"] > threshold, "signal"] = 1
        df_temp.loc[df_temp["rsi_momentum"] < -threshold, "signal"] = -1

        df_temp["return"] = df_temp["close"].pct_change().shift(-1)
        df_temp["strategy_return"] = df_temp["signal"] * df_temp["return"]

        hit_rate = (np.sign(df_temp["signal"]) == np.sign(df_temp["return"])).mean()
        sharpe = df_temp["strategy_return"].mean() / df_temp["strategy_return"].std()
        total_signals = df_temp["signal"].abs().sum()

        results_rsi.append({
            "symbol": symbol,
            "lookback": lookback,
            "threshold": round(threshold, 2),
            "hit_rate": round(hit_rate, 3),
            "sharpe_ratio": round(sharpe, 2),
            "signals": int(total_signals)
        })

# Present results as a DataFrame
results_rsi_df = pd.DataFrame(results_rsi)
results_rsi_df

,symbol,lookback,threshold,hit_rate,sharpe_ratio,signals
0,BTCUSDT,5,25.50,0.028,0.03,168
1,BTCUSDT,7,18.58,0.026,0.01,168
2,BTCUSDT,10,13.09,0.025,-0.01,168
3,BTCUSDT,14,9.50,0.025,-0.01,168
4,BTCUSDT,21,6.39,0.026,-0.00,167
5,ETHUSDT,5,24.40,0.026,-0.02,168
6,ETHUSDT,7,17.82,0.024,-0.04,168
7,ETHUSDT,10,12.43,0.025,-0.03,168
8,ETHUSDT,14,8.89,0.025,-0.04,168
9,ETHUSDT,21,5.97,0.024,-0.03,167


### 2. MACD

In [15]:
results_macd = []

for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_3min_7days.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index()

    # Calculate MACD and MACD Signal Line
    macd = MACD(close=df["close"], window_slow=26, window_fast=12, window_sign=9)
    df["macd"] = macd.macd()
    df["macd_signal"] = macd.macd_signal()
    df["macd_diff"] = df["macd"] - df["macd_signal"]

    # Generate signal based on MACD crossover
    df["signal"] = 0
    df.loc[(df["macd_diff"] > 0) & (df["macd_diff"].shift(1) <= 0), "signal"] = 1
    df.loc[(df["macd_diff"] < 0) & (df["macd_diff"].shift(1) >= 0), "signal"] = -1

    # Simulate returns
    df["return"] = df["close"].pct_change().shift(-1)
    df["strategy_return"] = df["signal"] * df["return"]

    # Evaluate performance
    hit_rate = (np.sign(df["signal"]) == np.sign(df["return"])).mean()
    sharpe = df["strategy_return"].mean() / df["strategy_return"].std()
    total_signals = df["signal"].abs().sum()

    results_macd.append({
        "symbol": symbol,
        "hit_rate": round(hit_rate, 3),
        "sharpe_ratio": round(sharpe, 2),
        "signals": int(total_signals)
    })

# Present results
results_macd_df = pd.DataFrame(results_macd)
results_macd_df

,symbol,hit_rate,sharpe_ratio,signals
0,BTCUSDT,0.044,0.04,256
1,ETHUSDT,0.037,-0.04,280
2,XRPUSDT,0.045,-0.02,273


### 3. Stochastic Oscillator

In [19]:
results_stoch = []

for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_3min_7days.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index()

    # Calculate Stochastic Oscillator
    stoch = StochasticOscillator(high=df["high"], low=df["low"], close=df["close"], window=14, smooth_window=3)
    df["stoch_k"] = stoch.stoch()
    df["stoch_d"] = stoch.stoch_signal()

    # Generate signal: Buy when %K crosses above %D from oversold, Sell when %K crosses below %D from overbought
    df["signal"] = 0
    df.loc[(df["stoch_k"] < 20) & (df["stoch_k"] > df["stoch_d"]) & (df["stoch_k"].shift(1) <= df["stoch_d"].shift(1)), "signal"] = 1
    df.loc[(df["stoch_k"] > 80) & (df["stoch_k"] < df["stoch_d"]) & (df["stoch_k"].shift(1) >= df["stoch_d"].shift(1)), "signal"] = -1

    # Simulate returns
    df["return"] = df["close"].pct_change().shift(-1)
    df["strategy_return"] = df["signal"] * df["return"]

    # Evaluate
    hit_rate = (np.sign(df["signal"]) == np.sign(df["return"])).mean()
    sharpe = df["strategy_return"].mean() / df["strategy_return"].std()
    total_signals = df["signal"].abs().sum()

    results_stoch.append({
        "symbol": symbol,
        "hit_rate": round(hit_rate, 3),
        "sharpe_ratio": round(sharpe, 2),
        "signals": int(total_signals)
    })

# Display results
results_stoch_df = pd.DataFrame(results_stoch)
results_stoch_df

,symbol,hit_rate,sharpe_ratio,signals
0,BTCUSDT,0.045,0.01,266
1,ETHUSDT,0.032,-0.02,206
2,XRPUSDT,0.046,-0.01,239


### 4.CCI

In [23]:
results_cci = []

for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_3min_7days.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index()

    # Calculate Commodity Channel Index (CCI)
    cci = CCIIndicator(high=df["high"], low=df["low"], close=df["close"], window=20)
    df["cci"] = cci.cci()

    # Generate signals based on classic CCI strategy
    # Buy when CCI crosses above -100; Sell when CCI crosses below +100
    df["signal"] = 0
    df.loc[(df["cci"] > -100) & (df["cci"].shift(1) <= -100), "signal"] = 1
    df.loc[(df["cci"] < 100) & (df["cci"].shift(1) >= 100), "signal"] = -1

    # Simulate returns
    df["return"] = df["close"].pct_change().shift(-1)
    df["strategy_return"] = df["signal"] * df["return"]

    # Evaluate
    hit_rate = (np.sign(df["signal"]) == np.sign(df["return"])).mean()
    sharpe = df["strategy_return"].mean() / df["strategy_return"].std()
    total_signals = df["signal"].abs().sum()

    results_cci.append({
        "symbol": symbol,
        "hit_rate": round(hit_rate, 3),
        "sharpe_ratio": round(sharpe, 2),
        "signals": int(total_signals)
    })

# Display results
results_cci_df = pd.DataFrame(results_cci)
results_cci_df

,symbol,hit_rate,sharpe_ratio,signals
0,BTCUSDT,0.047,-0.02,323
1,ETHUSDT,0.042,-0.03,307
2,XRPUSDT,0.057,0.01,317


### 5. Momentum

In [26]:
results_momentum = []

for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_3min_7days.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index()

    # Compute Momentum (Rate of Change)
    roc = ROCIndicator(close=df["close"], window=20)
    df["momentum"] = roc.roc()

    # Generate signal
    df["signal"] = 0
    df.loc[df["momentum"] > 0.2, "signal"] = 1
    df.loc[df["momentum"] < -0.2, "signal"] = -1

    # Simulate returns
    df["return"] = df["close"].pct_change().shift(-1)
    df["strategy_return"] = df["signal"] * df["return"]

    # Evaluate
    hit_rate = (np.sign(df["signal"]) == np.sign(df["return"])).mean()
    sharpe = df["strategy_return"].mean() / df["strategy_return"].std()
    total_signals = df["signal"].abs().sum()

    results_momentum.append({
        "symbol": symbol,
        "hit_rate": round(hit_rate, 3),
        "sharpe_ratio": round(sharpe, 2),
        "signals": int(total_signals)
    })

# Display results
results_momentum_df = pd.DataFrame(results_momentum)
results_momentum_df

,symbol,hit_rate,sharpe_ratio,signals
0,BTCUSDT,0.260,0.02,1739
1,ETHUSDT,0.369,-0.01,2561
2,XRPUSDT,0.352,-0.00,2432


In [ ]:

class BaseSignalStrategy(bt.Strategy):
    params = (('record_signals', True),)

    def __init__(self):
        self.signal = 0  # 当前信号
        self.signals = []  # 记录历史信号

    def next(self):
        if self.record_signals:
            self.signals.append({
                'date': self.data.datetime.date(0),
                'signal': self.signal
            })

    def get_signals_df(self):
        return pd.DataFrame(self.signals).set_index('date')



In [ ]:

# --------------------------
# 独立指标策略（按需扩展）
# --------------------------
class RSIStrategy(BaseSignalStrategy):
    params = (('rsi_period', 14), ('oversold', 30), ('overbought', 70))

    def __init__(self):
        super().__init__()
        self.rsi = bt.indicators.RSI(self.data.close, period=self.p.rsi_period)

    def next(self):
        if self.rsi < self.p.oversold:
            self.signal = 1  # 买入信号
        elif self.rsi > self.p.overbought:
            self.signal = -1  # 卖出信号
        else:
            self.signal = 0
        super().next()



In [ ]:

class CCIStrategy(BaseSignalStrategy):
    params = (('cci_period', 20), ('oversold', -100), ('overbought', 100))

    def __init__(self):
        super().__init__()
        self.cci = bt.indicators.CCI(self.data, period=self.p.cci_period)

    def next(self):
        if self.cci < self.p.oversold:
            self.signal = 1
        elif self.cci > self.p.overbought:
            self.signal = -1
        else:
            self.signal = 0
        super().next()


In [ ]:


class MACDStrategy(BaseSignalStrategy):
    params = (('fast', 12), ('slow', 26), ('signal_period', 9))

    def __init__(self):
        super().__init__()
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.p.fast,
            period_me2=self.p.slow,
            period_signal=self.p.signal_period
        )
        self.crossover = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)

    def next(self):
        if self.crossover > 0:
            self.signal = 1  # 金叉买入
        elif self.crossover < 0:
            self.signal = -1  # 死叉卖出
        else:
            self.signal = 0
        super().next()



In [ ]:
class StochasticStrategy(bt.Strategy):
    params = (
        ('k_period', 14),    # %K计算周期
        ('d_period', 3),     # %D计算周期（%K的SMA）
        ('overbought', 80),  # 超买阈值
        ('oversold', 20),    # 超卖阈值
        ('use_trend_filter', True),  # 是否使用趋势过滤
    )

    def __init__(self):
        # 计算指标
        self.stoch = bt.indicators.Stochastic(
            self.data,
            period=self.p.k_period,
            period_dfast=self.p.d_period,
            upperband=self.p.overbought,
            lowerband=self.p.oversold
        )
        self.crossover = bt.indicators.CrossOver(self.stoch.lines.percK, self.stoch.lines.percD)
        
        # 趋势过滤（可选）
        if self.p.use_trend_filter:
            self.sma200 = bt.indicators.SMA(self.data.close, period=200)

    def next(self):
        # 生成信号（1=买入，-1=卖出，0=无信号）
        signal = 0
        
        # 多头条件
        if (self.stoch.lines.percK[0] < self.p.oversold and 
            self.crossover > 0 and 
            (not self.p.use_trend_filter or self.data.close[0] > self.sma200[0])):
            signal = 1
        
        # 空头条件
        elif (self.stoch.lines.percK[0] > self.p.overbought and 
              self.crossover < 0 and 
              (not self.p.use_trend_filter or self.data.close[0] < self.sma200[0])):
            signal = -1
        
        # 记录信号（用于后续机器学习组合）
        self.signal = signal

In [ ]:

class MOMStrategy(BaseSignalStrategy):
    params = (('mom_period', 10), ('threshold', 0))

    def __init__(self):
        super().__init__()
        self.mom = bt.indicators.Momentum(self.data.close, period=self.p.mom_period)

    def next(self):
        if self.mom > self.p.threshold:
            self.signal = 1  # 动量向上
        elif self.mom < -self.p.threshold:
            self.signal = -1  # 动量向下
        else:
            self.signal = 0
        super().next()